# Harinero Tanda Creator - Example

This notebook demonstrates how to use the machine learning components of Harinero Tanda Creator to:
- Train similarity models on song pairs
- Make similarity predictions
- Generate tandas using ML-based suggestions

First, let's import our required components and set up the environment.

In [ ]:
from harinero.core.database import SongFinder
from harinero.ml.preprocessing import Preprocessor 
from harinero.ml.models import KerasSongModel
from harinero.core import HarineroTandaCreator
from harinero.utils import load_config
import pandas as pd
import numpy as np

# Load configuration
config = load_config("/path/to/config.json")

# Initialize components
creator = HarineroTandaCreator(config)
print("Harinero ML components initialized")

## 1. Training a Similarity Model

We'll train a model to recognize similar songs. The training process:
1. Selects a reference song
2. Finds similar/dissimilar examples
3. Trains the model on these pairs

In [ ]:
# Train on a specific song
reference_song_id = 1  # La Cumparsita by D'Arienzo

try:
    # Train the model
    history, results = creator.train_on_song(reference_song_id)
    
    print("\nTraining Results:")
    print(f"Final Loss: {results[0]:.4f}")
    print(f"Final Accuracy: {results[1]:.4f}")
    
    # Plot training history
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.legend()
    plt.show()
    
except Exception as e:
    print(f"Error during training: {e}")

## 2. Making Similarity Predictions

Now that we have a trained model, let's use it to predict similarities between songs.

In [ ]:
# Get predictions for a song
try:
    # Get similarity scores
    similarity_scores = creator.predict_on_song(reference_song_id)
    print(f"Average similarity score: {similarity_scores:.4f}")
    
    # Find similar songs with their probabilities
    similar_songs_df = creator.get_similar_songs_with_probabilities(
        song_id=reference_song_id,
        year_span=5  # Look within 5 years
    )
    
    print("\nTop 5 Most Similar Songs:")
    display(similar_songs_df.head())
    
except Exception as e:
    print(f"Error during prediction: {e}")

## 3. Creating ML-Assisted Tandas

We can use our similarity predictions to help create tandas by suggesting songs that work well together.

In [ ]:
# Create a tanda using ML suggestions
try:
    # Get top similar songs
    similar_songs_df = creator.get_similar_songs_with_probabilities(
        song_id=reference_song_id,
        year_span=5
    )
    
    # Select top 4 songs for the tanda
    tanda_song_ids = [reference_song_id] + similar_songs_df.head(3)['song_id'].tolist()
    
    # Create the tanda
    tanda = creator.create_tanda(tanda_song_ids, tanda_number=1)
    
    print("Created ML-assisted tanda:")
    print(tanda)
    
except Exception as e:
    print(f"Error creating tanda: {e}")

## 4. Analyzing ML-Based Tandas

Let's analyze the musical characteristics of our ML-generated tanda.

In [ ]:
# Generate overview and details
try:
    # Get tanda overview
    overview_df = creator.generate_tanda_overview()
    print("Tanda Overview:")
    display(overview_df)
    
    # Get detailed view
    detail_df = creator.generate_tanda_detail()
    print("\nTanda Details:")
    display(detail_df)
    
    # Plot musical progression
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.plot(detail_df['tempo'], marker='o')
    plt.title('Tempo Progression')
    plt.ylabel('BPM')
    
    plt.subplot(1, 3, 2)
    plt.plot(detail_df['beat_strength'], marker='o')
    plt.title('Beat Strength Progression')
    
    plt.subplot(1, 3, 3)
    plt.plot(detail_df['brightness'], marker='o')
    plt.title('Brightness Progression')
    
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error analyzing tanda: {e}")

In [ ]:
tanda.play()

## Best Practices for ML-Based Tanda Creation

1. **Model Training**:
   - Use representative reference songs
   - Ensure diverse negative examples
   - Monitor training metrics carefully
   - Save best performing model checkpoints

2. **Similarity Predictions**:
   - Consider temporal context (year_span)
   - Use appropriate thresholds
   - Validate predictions against expert knowledge
   - Consider multiple similarity aspects

3. **Tanda Creation**:
   - Balance ML suggestions with traditional rules
   - Verify musical progression
   - Check audio quality consistency
   - Consider dance floor energy

4. **Performance**:
   - Cache model predictions
   - Batch process similar songs
   - Pre-compute common similarities
   - Use appropriate batch sizes

5. **Quality Control**:
   - Validate ML suggestions. You can do this by listening by using play() or playex() from song objects.
   - Keep track of model performance
   - Document unusual cases